In [1]:
import pandas as pd
import seaborn as sns; sns.set('notebook')
import matplotlib.pyplot as plt
params = {'legend.fontsize': 'large',
          'figure.figsize': (16, 9),
         'axes.labelsize': 'large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'medium',
         'ytick.labelsize':'medium'}
plt.rcParams.update(params)

In [10]:
df = pd.read_csv('resnet_inference_test.csv', index_col=0)
df.mean()

exit          2.726000
prediction    4.428000
target        4.500000
correct       0.972000
score         0.089660
time          0.027784
dtype: float64

In [7]:
data = df.to_numpy()

In [13]:
df = pd.DataFrame(data[1:], columns=['exit', 'prediction', 'target', 'correct', 'score', 'time'])

In [14]:
df.mean()

exit          2.731463
prediction    4.436874
target        4.509018
correct       0.971944
score         0.088206
time          0.025892
dtype: float64

In [15]:
len(df.loc[df['exit']==0])

37

In [20]:
len(df.loc[(df['exit']==0) & (df['correct'] == False)])

2

In [16]:
len(df.loc[df['exit']==1])

9

In [17]:
len(df.loc[df['exit']==2])

5

In [21]:
len(df.loc[(df['exit']==3) & (df['correct'] == False)])

12

In [22]:
len(df.loc[(df['correct'] == False)])

14